# Import necessary libraries

In [32]:
import numpy as np
import time
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load and preprocess MNIST

In [33]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1)).astype('float32') / 255
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1)).astype('float32') / 255
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10) 

# Build a simple CNN model

In [34]:
def create_model():
    input = Input(shape=(28, 28, 1))
    x = Flatten()(input)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    output = Dense(10, activation='softmax')(x)
    model = Model(inputs=input, outputs=output)
    return model

# Custom training using tf.GradientTape

In [40]:
def train_with_tape():
    model = create_model()
    optimizer = tf.keras.optimizers.Adam()
    loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    acc_metric = tf.keras.metrics.CategoricalAccuracy()

    start = time.time()
    for epoch in range(5):
        print(f"Epoch {epoch+1}/5")
        acc_metric.reset_state()
        for i in range(0, len(x_train), 32):
            x_batch = x_train[i:i+32]
            y_batch = y_train[i:i+32]  # Already one-hot
            with tf.GradientTape() as tape:
                logits = model(x_batch, training=True)
                loss = loss_fn(y_batch, logits)
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
            acc_metric.update_state(y_batch, logits)
        print(f"Train accuracy: {acc_metric.result().numpy():.4f}")
    end = time.time()

    # Test accuracy
    logits_test = model(x_test, training=False)
    test_acc = tf.keras.metrics.categorical_accuracy(y_test, logits_test)
    final_acc = tf.reduce_mean(test_acc).numpy()

    return final_acc, end - start


# Training using model.fit

In [36]:
def train_with_fit():
    model = create_model()
    model.compile(
        optimizer='adam',
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )
    start = time.time()
    model.fit(x_train, y_train, epochs=5, batch_size=32, verbose=0)
    end = time.time()
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    print("Test accuracy (fit):", test_acc)
    return test_acc, end - start    

In [ ]:
acc_tape, time_tape = train_with_tape()
acc_fit, time_fit = train_with_fit()


print("\n Final Comparison:")
print(f"tf.GradientTape(): Accuracy = {acc_tape:.4f}, Time = {time_tape:.2f} sec")
print(f"model.fit():       Accuracy = {acc_fit:.4f}, Time = {time_fit:.2f} sec")

Epoch 1/5


/home/rayhanul/.local/lib/python3.11/site-packages/keras/src/backend/tensorflow/nn.py:675: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Train accuracy: 0.9253
Epoch 2/5
Train accuracy: 0.9668
Epoch 3/5
Train accuracy: 0.9775
Epoch 4/5
Train accuracy: 0.9834
Epoch 5/5
